In [1]:
!pip install torch transformers pillow

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

Welcome to the Image Caption Generator!
Enter the path to an image file (e.g., 'path/to/image.jpg'). Type 'quit' to exit.



Enter image path:  images/cat_window.jpg



Generated Caption: there is a cat that is standing on its hind legs looking out a window



Enter image path:  quit


Exiting the caption generator. Goodbye!


In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

# --- Step 1: Load the Pre-trained Model and Processor ---
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# --- Step 2: Function to Generate Caption ---
def generate_caption(image_path):
    """
    Generate a caption for an image file.
    
    Args:
        image_path (str): Path to the image file (e.g., 'dog_beach.jpg').
    
    Returns:
        str: Generated caption.
    """
    try:
        # Load and preprocess the image
        image = Image.open(image_path).convert("RGB")
        inputs = processor(images=image, return_tensors="pt").to(device) #here to device is used to move the image to GPU if available
        
        # Generate caption
        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                max_length=50,  # Limit caption length
                num_beams=5,    # Beam search for better quality
                temperature=1.0 # Default creativity
            )
        caption = processor.decode(output_ids[0], skip_special_tokens=True)
        return caption
    except Exception as e:
        return f"Error generating caption: {e}"

# --- Step 3: Main Interface ---
print("Welcome to the Image Caption Generator!")
print("Enter the path to an image file (e.g., 'path/to/image.jpg'). Type 'quit' to exit.")
while True:
    image_path = input("\nEnter image path: ")
    if image_path.lower() == 'quit':
        print("Exiting the caption generator. Goodbye!")
        break
    
    caption = generate_caption(image_path)
    print("\nGenerated Caption:", caption)